In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pylab as plt

Let's try to solve the wave equation using PINNs, 
$$
\begin{matrix}
\partial_t u + \partial_x u =0, & x \in [0,1], t \in [0,1]\\
u = 0 & x=0\\
u(x,t=0) = \exp(-4(x-0.3)^2) & x \in [0.2,0.5]\\
u(x,t=0) = 0  & x<0.1\textrm{ or }x>0.5\\
\end{matrix}
$$


In [ ]:
#Create a grid of points for the full domain and separate them into
#interior, boundary, and initial condition points

#creates a 2d grid in [0,1]x[0,1] with 100 points in each direction. W
xt_ = np.mgrid[0:1:100j,0:1:100j].astype('float32')

In [ ]:
#let's reorder this so the points are all in the first dimensino
xt_.shape

In [ ]:
#all points are now in the first dimension. Let's treat the first dimension as 
#space and the second as time
xt = np.reshape(np.transpose(xt_,(1,2,0)),(-1,2))

In [ ]:
#these points are in the boundary
isbc = xt[:,0]<1e-4
#these points are at the initial condition
isic = xt[:,1]<1e-4

#these points are in the interior
isinterior = np.logical_and(np.logical_not(isic),np.logical_not(isbc))

xt_bc = xt[isbc]
xt_ic = xt[isic]
xt_interior = xt[isinterior]

In [ ]:
#plot of collocation points in space time
plt.scatter(*xt_bc.T)
plt.scatter(*xt_ic.T)
plt.scatter(*xt_interior.T)

In [ ]:
#Makes a neural network to represent the solution
u = tf.keras.models.Sequential([
    tf.keras.layers.Input(2,),
    tf.keras.layers.Dense(10,activation='elu'),
    tf.keras.layers.Dense(10,activation='elu'),
    tf.keras.layers.Dense(10,activation='elu'),
    tf.keras.layers.Dense(10,activation='elu'),
    tf.keras.layers.Dense(1),
])
u.summary()

In [ ]:
#our 'solution' before training
plt.imshow(np.reshape(u(xt),(100,100)).T,extent=[0,1,0,1],origin='lower')
plt.xlabel('$x$')
plt.ylabel('$t$')
cb = plt.colorbar()
cb.set_label('$u$')

In [ ]:
#PDE residual. A couple tricks here,
#1) tape.gradient actually computes \sum(u_) w.r.t. xt. Since each u_ at a point only 
#depends on the (x,t) at that point, computing the gradient of the sum effectively 
#computes (\partial_x u,\partial_t u) at each point. 
#2) We take the sum over the last axis to get (\partial_x u + \partial_t u) at each point
#This is the per collocation point residual. We take the some of squares to get the 
#residual for the interior of the domain

#Note that you can pass numpy tensors to a tf.function and still use GradientTape against them inside

@tf.function
def PDE_res(xt):
    with tf.GradientTape() as tape:
        tape.watch(xt)
        u_ = u(xt)
    return tf.reduce_sum(tf.reduce_sum(tape.gradient(u_,xt),axis=-1)**2)
PDE_res(xt_interior)

In [ ]:
#At t=0, we'd like to match the function

def IC(x):
    return \
      (1. - tf.cast(x<.1,tf.float32)) \
    * (1. - tf.cast(x>.5,tf.float32)) \
    * tf.cast(x>=.1,tf.float32)*tf.cast(x<=.5,tf.float32) * (tf.exp(-64*(x-0.3)**2))
    
plt.plot(xt_ic[:,0],IC(xt_ic[:,0]))
plt.xlabel('x')
plt.ylabel('$u(x,t=0)$')

In [ ]:
#IC residual
def IC_res(xt):
    return tf.reduce_sum((IC(xt[:,0]) - u(xt)[:,0])**2)

#Similarly, we'd like the solution to equal 0 at the left BC.
#We'll penalize against nonzero sum of squares

def BC_res(xt):
    return tf.reduce_sum(u(xt)**2)
    

In [ ]:
#Combined residual. We can also add weighting to each term

alpha= 100.
beta = 1.
@tf.function
def res():
    return PDE_res(xt_interior) + alpha*IC_res(xt_ic) + beta*BC_res(xt_bc)



In [ ]:
opt = tf.keras.optimizers.Adam(1e-3)
@tf.function
def train():
    with tf.GradientTape() as tape:
        tape.watch(u.trainable_variables)
        res_ = res()
    grad_ = tape.gradient(res_,u.trainable_variables)
    opt.apply_gradients(zip(grad_,u.trainable_variables))
    return res_

In [ ]:
for _ in range(1000):
    print(train().numpy())

In [ ]:
#our solution after training
plt.imshow(np.reshape(u(xt),(100,100)).T,extent=[0,1,0,1],origin='lower')
plt.xlabel('$x$')
plt.ylabel('$t$')
cb = plt.colorbar()
cb.set_label('$u$')

# PINNs inverse problem

This time, let's infer the wave speed, $c$,

$$
\begin{matrix}
\partial_t u + c\partial_x u =0, & x \in (0,1], t \in (0,1]\\
u = 0 & x=0\\
u(x,t=0) = \exp(-4(x-0.3)^2) & x \in [0.2,0.5]\\
u(x,t=0) = 0  & x<0.1\textrm{ or }x>0.5\\
\end{matrix}
$$

given sythetic data (analytical solution using $c=0.5$ at some random points)

In [ ]:
#Makes a neural network to represent the solution
u = tf.keras.models.Sequential([
    tf.keras.layers.Input(2,),
    tf.keras.layers.Dense(10,activation='elu'),
    tf.keras.layers.Dense(10,activation='elu'),
    tf.keras.layers.Dense(10,activation='elu'),
    tf.keras.layers.Dense(10,activation='elu'),
    tf.keras.layers.Dense(1),
])
u.summary()

In [ ]:
#IC residual

def IC_res(xt):
    return tf.reduce_sum((IC(xt[:,0]) - u(xt)[:,0])**2)

#Similarly, we'd like the solution to equal 0 at the left BC.
#We'll penalize against nonzero sum of squares

def BC_res(xt):
    return tf.reduce_sum(u(xt)**2)

def data_res(xt_random):
    u_true = IC(xt[:,0]-.5*xt[:,1])
    return tf.reduce_sum((u(xt_random)[...,0] - u_true)**2)

# Speed we're trying to infer. Initialize at 1
c_tf = tf.Variable(1.)
@tf.function
def PDE_res(xt,c_tf):
    with tf.GradientTape() as tape:
        tape.watch(xt)
        u_ = u(xt)
    return tf.reduce_sum(tf.reduce_sum(tape.gradient(u_,xt)*[c_tf,1.],axis=-1)**2)


In [ ]:
#Combined residual. We can also add weighting to each term

alpha= 1.
beta = 1.
gamma = 10.
@tf.function
def res(c_tf):
    xt_random = np.random.uniform(0,1,(100,2)).astype('float32')
    return PDE_res(xt_interior,c_tf) + alpha*IC_res(xt_ic) + beta*BC_res(xt_bc) + gamma*data_res(xt)

In [ ]:
opt = tf.keras.optimizers.Adam(1e-3)
@tf.function
def train():
    with tf.GradientTape() as tape:
        tape.watch(u.trainable_variables+[c_tf])
        res_ = res(c_tf)
    grad_ = tape.gradient(res_,u.trainable_variables+[c_tf])
    opt.apply_gradients(zip(grad_,u.trainable_variables+[c_tf]))
    return res_

In [ ]:
for _ in range(1000):
    print(train().numpy())

In [ ]:
print('PINNs inferred speed, ',c_tf)

In [ ]:
#our solution after training
plt.imshow(np.reshape(u(xt),(100,100)).T,extent=[0,1,0,1],origin='lower')
plt.xlabel('$x$')
plt.ylabel('$t$')
cb = plt.colorbar()
cb.set_label('$u$')

In [ ]:
u_true = IC(xt[:,0]-.5*xt[:,1])
plt.scatter(*xt.T,c=u_true);plt.colorbar()